# math

In [ ]:
open testing
open rust_operators

()



In [ ]:
inl types () =
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"num_complex::Complex<$0>\")>] type num_complex_Complex<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"pyo3::types::PyModule\")>] type pyo3_types_PyModule = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"pyo3::Bound<$0>\")>] type pyo3_Bound<'T> = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"pyo3::Python\")>] type pyo3_Python = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"pyo3::PyAny\")>] type pyo3_PyAny = class end"
    global "[<Fable.Core.Erase; Fable.Core.Emit(\"pyo3::PyErr\")>] type pyo3_PyErr = class end"

()



In [ ]:
inl types () =
    rust.types ()
    sm'.types ()
    types ()

()



## complex

In [ ]:
nominal complex t = $"num_complex_Complex<`t>"
nominal bound t = $"pyo3_Bound<`t>"
nominal python = $"pyo3_Python"
nominal pymodule = $"pyo3_types_PyModule"
nominal pyany = $"pyo3_PyAny"
nominal pyerr = $"pyo3_PyErr"

inl complex forall t. ((re : t), (im : t)) : complex t =
    inl re = join re
    inl im = join im
    !\($'"num_complex::Complex::new(!re, !im)"')

()



In [ ]:
// // test
// // rust=

types ()

complex (0f64, 0f64)
|> sm'.format'
|> sm'.from_std_string
|> _assert_eq "0+0i"

.rs output:




.fsx:
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>] type Func0<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>] type Func0<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>] type Box<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>] type Dyn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>] type Fn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>] type FnUnit = class end
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>] type FnOnce<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>] type ActionFn2<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("impl $0")>] type Impl<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("mut $0")>] type Mut<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&$0")>] type Ref<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&'static $0")>] type StaticRef<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("MutCell<$0>")>] type MutCe

## run_test

In [ ]:
inl re forall t. (c : complex t) : t =
    inl c = join c
    !\($'"!c.re"')

inl im forall t. (c : complex t) : t =
    inl c = join c
    !\($'"!c.im"')

inl complex_unbox forall t. (c : complex t) =
    c |> re, c |> im

inl (~.^) c = complex c

inl complex_eq forall t. (a : complex t) (b : complex t) : bool =
    !\($'"!a == !b"')

inl (.=) a b = complex_eq a b

instance equable complex t = complex_eq

inl complex_add forall t. (a : complex t) (b : complex t) : complex t =
    !\($'"!a + !b"')

inl (.+) a b = complex_add a b

inl complex_sub forall t. (a : complex t) (b : complex t) : complex t =
    !\($'"!a - !b"')

inl (.-) a b = complex_sub a b

inl complex_mult forall t. (a : complex t) (b : complex t) : complex t =
    !\($'"!a * !b"')

inl (.*) a b = complex_mult a b

inl complex_div forall t. (a : complex t) (b : complex t) : complex t =
    !\($'"!a / !b"')

inl (./) a b = complex_div a b

inl powc forall t. (s : complex t) (c : complex t) : complex t =
    inl c = join c
    inl s = join s
    !\($'"num_complex::Complex::powc(!c, !s)"')

inl (.**) a b = powc b a

inl complex_sin forall t. (c : complex t) : complex t =
    !\($'"!c.sin()"')

inl conj forall t. (c : complex t) : complex t =
    !\($'"!c.conj()"')

()



## eval

In [ ]:
inl module_from_code (py : python) (code : string) : resultm.result' (bound pymodule) pyerr =
    inl py = join py
    inl code = code |> sm'.as_str
    !\($'"pyo3::types::PyModule::from_code_bound(!py, !code, \\"\\", \\"\\")"')

inl use_pyanymethods () =
    global "Fable.Core.RustInterop.emitRustExpr () \");\nuse pyo3::prelude::PyAnyMethods;\n//\""

inl getattr (attr : string) (module : bound pymodule) : resultm.result' (bound pyany) pyerr =
    inl attr = attr |> sm'.as_str
    inl module = join module
    use_pyanymethods ()
    !\($'"!module.getattr(!attr)"')

inl call forall t. (args : t) (module : bound pyany) : resultm.result' (bound pyany) pyerr =
    inl args = join args
    inl module = join module
    !\($'"pyo3::prelude::PyAnyMethods::call(&!module, (*(*!args).0, ()), None)"')

inl extract forall t. (result : bound pyany) : resultm.result' t pyerr =
    inl result = join result
    use_pyanymethods ()
    !\($'"!result.extract()"')

inl eval py code args =
    inl code =
        code
        |> module_from_code py
        |> resultm.unwrap'
    inl fn =
        code
        |> getattr "fn"
        |> fun x => x : _ _ pyerr
        |> resultm.unwrap'

    fn
    |> call args
    |> resultm.unwrap'
    |> extract
    |> fun x => x : _ _ pyerr
    |> resultm.unwrap'
    |> complex
    |> Ok
    |> fun x => x : _ _ pyerr
    |> resultm.box

inl gamma_ py s =
    inl code =
        ;[
            "import mpmath"
            "def fn(s, _):"
            "    s = complex(*s)"
            "    s = mpmath.gamma(s)"
            "    return (s.real, s.imag)"
        ]
    inl code = (a code : _ i32 _) |> sm'.concat_array_trailing "\n"

    inl s = new_pair (s |> re) (s |> im)
    inl args = new_pair s ()

    eval py code args

inl zeta_ py s =
    inl code =
        ;[
            "import mpmath"
            "def fn(s, _):"
            "    s = complex(*s)"
            "    try:"
            "        s = mpmath.zeta(s)"
            "    except ValueError as e:"
            "        if s.real == 1:"
            "            s = complex(float('inf'), 0)"
            "    return (s.real, s.imag)"
        ]
    inl code = (a code : _ i32 _) |> sm'.concat_array_trailing "\n"
    
    inl s = new_pair (s |> re) (s |> im)
    inl args = new_pair s ()

    eval py code args

()



## zeta

In [ ]:
inl rec zeta (gamma : complex f64 -> complex f64) (s : complex f64) : complex f64 =
    join
        if re s > 1 then
            am.init 10000i32 id
            |> fun x => x : a i32 _
            |> am.fold
                fun acc n =>
                    acc .+ .^(1, 0) ./ (.^(f64 n, 0) .** s)
                .^(0, 0)
        elif s = .^(-1, 0) then
            .^(-1 / 12, 0)
        else
            inl one_minus_s = .^(1 - re s, -(im s))
            inl mirror_term =
                if re one_minus_s > 1
                then zeta gamma one_minus_s
                else .^(0, 0)
            inl gamma_term = gamma (.^(1, 0) .- s)
            inl sin_term = .^(pi, 0) .* s ./ .^(2, 0) |> complex_sin
            inl reflection_formula =
                .^(2, 0) .* (.^(pi, 0) .** s) .* sin_term .* gamma_term .* mirror_term
            reflection_formula

inl zeta (gamma : complex f64 -> complex f64) (s : complex f64) : complex f64 =
    .^(0, 0)

()



## run_test

In [ ]:
inl run_test closure_fix (fn : (complex f64 -> complex f64) * (complex f64 -> complex f64) -> ()) =
    inl fn_ (py : python) : resultm.result' () pyerr =
        inl gamma__ = fun (s : complex f64) =>
            inl s = gamma_ py s |> resultm.unwrap'
            s
        inl zeta__ = fun (s : complex f64) =>
            inl result = zeta_ py s |> resultm.unwrap'

            inl z = zeta gamma__ s

            !\($'"println\!(\\\"zeta / s: {:?} / result: {:?} / z: {:?}\\\", !s, !result, !z)"')

    //             re result - re x |> abs
    //             |> _assert_lt 0.001
    //             
    //             im result - im x |> abs
    //             |> _assert_lt 0.001

            result
        fn (zeta__, gamma__)
        
        Ok ()
        |> resultm.box
    
    join
        !\($'"pyo3::prepare_freethreaded_python()"') : ()

        !\($'"let __result = pyo3::Python::with_gil(|py| -> pyo3::PyResult<()> { //"')

        inl x = fn_
        let x' = x (!\($'"py"') : python)
        inl x' = join x'
        
        x' |> rust.fix_closure closure_fix

        (!\($'"__result"') : _ () pyerr)
        |> resultm.unwrap'

()



## test_zeta_at_known_values_

In [ ]:
inl test_zeta_at_known_values_ () = run_test (3u8, 2u8) fun zeta, gamma =>
    ;[
        .^(2, 0), pi ** 2 / 6
        .^(-1, 0), -1 / 12
    ]
    |> fun x => a x : _ i32 _
    |> am.iter fun s, e =>
        inl result = zeta s

        result |> im |> _assert_eq 0
        re result - e |> abs |> _assert_lt 0.0001

()



In [ ]:
// // test
// // rust=
// // print_code=false


types ()
test_zeta_at_known_values_ ()

zeta / s: Complex { re: 2.0, im: 0.0 } / result: Complex { re: 1.6449340668482264, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -1.0, im: 0.0 } / result: Complex { re: -0.08333333333333333, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }


## test_zeta_at_2_minus2

In [ ]:
inl test_zeta_at_2_minus2 () = run_test (6u8, 5u8) fun zeta, gamma =>
    inl s = .^(2, -2)
    inl result = zeta s

    (re result - 0.8673) |> abs |> _assert_lt 0.001
    (im result - 0.2750) |> abs |> _assert_lt 0.001

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_zeta_at_2_minus2 ()

zeta / s: Complex { re: 2.0, im: -2.0 } / result: Complex { re: 0.8673518296359931, im: 0.27512723880785767 } / z: Complex { re: 0.0, im: 0.0 }


## test_trivial_zero_at_negative_even___

In [ ]:
inl test_trivial_zero_at_negative_even___ () = run_test (2u8, 1u8) fun zeta, gamma =>
    (join listm'.init_series -2f64 -40 -2)
    |> listm.iter fun n =>
        inl s = .^(n, 0)
        inl result = zeta s

        result |> re |> _assert_eq 0
        result |> im |> _assert_eq 0

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_trivial_zero_at_negative_even___ ()

zeta / s: Complex { re: -2.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -4.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -6.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -8.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -10.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -12.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -14.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -16.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -18.0, im: 0.0 } / result: Complex { re: 0.0, im: 0.0 } / z: Complex

## test_non_trivial_zero___

In [ ]:
inl test_non_trivial_zero___ () = run_test (3u8, 2u8) fun zeta, gamma =>
    ;[
        .^(0.5, 14.134725)
        .^(0.5, 21.022040)
        .^(0.5, 25.010857)
        .^(0.5, 30.424876)
        .^(0.5, 32.935062)
        .^(0.5, 37.586178)
    ]
    |> fun x => a x : _ i32 _
    |> am.iter fun x =>
            inl result = zeta x
            result |> re |> abs |> _assert_lt 0.0001
            result |> im |> abs |> _assert_lt 0.0001

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_non_trivial_zero___ ()

zeta / s: Complex { re: 0.5, im: 14.134725 } / result: Complex { re: 1.767429841384921e-8, im: -1.1102028930923147e-7 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.5, im: 21.02204 } / result: Complex { re: 8.984836054354555e-8, im: 4.007090847649033e-7 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.5, im: 25.010857 } / result: Complex { re: 2.6108713770239107e-7, im: -7.517500707346376e-7 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.5, im: 30.424876 } / result: Complex { re: -8.401240452550172e-8, im: -1.4097913401178508e-7 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.5, im: 32.935062 } / result: Complex { re: -3.0942393440028226e-7, im: 4.784577171292422e-7 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.5, im: 37.586178 } / result: Complex { re: -8.910186507947958e-8, im: -2.943780446402868e-7 } / z: Complex { re: 0.0, im: 0.0 }


## test_real_part_greater_than_one___

In [ ]:
inl test_real_part_greater_than_one___ () = run_test (3u8, 2u8) fun zeta, gamma =>
    inl points = ;[2; 3; 4; 5; 10; 20; 50]
    (a points : _ i32 _)
    |> am.iter fun point =>
        inl s = .^(point, 0)
        inl result = zeta s
        result |> re |> _assert_gt 0
        result |> im |> _assert_eq 0

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_real_part_greater_than_one___ ()

zeta / s: Complex { re: 2.0, im: 0.0 } / result: Complex { re: 1.6449340668482264, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 3.0, im: 0.0 } / result: Complex { re: 1.2020569031595942, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 4.0, im: 0.0 } / result: Complex { re: 1.0823232337111381, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 5.0, im: 0.0 } / result: Complex { re: 1.03692775514337, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 10.0, im: 0.0 } / result: Complex { re: 1.000994575127818, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 20.0, im: 0.0 } / result: Complex { re: 1.0000009539620338, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 50.0, im: 0.0 } / result: Complex { re: 1.0000000000000009, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }


## test_zeta_at_1___

In [ ]:
inl test_zeta_at_1___ () = run_test (6u8, 5u8) fun zeta, gamma =>
    inl s = .^(1, 0)
    inl result = zeta s
    result |> re |> _assert_eq limit.max
    result |> im |> _assert_eq 0

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_zeta_at_1___ ()

zeta / s: Complex { re: 1.0, im: 0.0 } / result: Complex { re: inf, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }


## test_symmetry_across_real_axis___

In [ ]:
inl test_symmetry_across_real_axis___ () = run_test (8u8, 7u8) fun zeta, gamma =>
    inl s = .^(2, 10)
    inl result_positive_im = zeta s
    inl result_negative_im = zeta .^(re s, -(im s))
    inl conj = result_negative_im |> conj
    result_positive_im |> re |> _assert_eq (conj |> re)
    result_positive_im |> im |> _assert_eq (conj |> im)

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_symmetry_across_real_axis___ ()

zeta / s: Complex { re: 2.0, im: 10.0 } / result: Complex { re: 1.1979825006741847, im: -0.07917049172052575 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 2.0, im: -10.0 } / result: Complex { re: 1.1979825006741847, im: 0.07917049172052575 } / z: Complex { re: 0.0, im: 0.0 }


## test_behavior_near_origin___

In [ ]:
inl test_behavior_near_origin___ () = run_test (6u8, 5u8) fun zeta, gamma =>
    inl s = .^(0.01, 0.01)
    inl result = zeta s
    result |> re |> _assert_lt limit.max
    result |> im |> _assert_lt limit.max

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_behavior_near_origin___ ()

zeta / s: Complex { re: 0.01, im: 0.01 } / result: Complex { re: -0.5091873433665667, im: -0.00939202213994577 } / z: Complex { re: 0.0, im: 0.0 }


## test_zeta_at_minus_1

In [ ]:
inl test_zeta_at_minus_1 () = run_test (6u8, 5u8) fun zeta, gamma =>
    inl s = .^(-1, 0)
    inl result = zeta s
    re result + 1 / 12 |> abs |> _assert_lt 0.0001
    result |> im |> _assert_eq 0

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_zeta_at_minus_1 ()

zeta / s: Complex { re: -1.0, im: 0.0 } / result: Complex { re: -0.08333333333333333, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }


## test_imaginary_axis

In [ ]:
inl test_imaginary_axis () = run_test (3u8, 2u8) fun zeta, gamma =>
    (join a ;[10; 20; 30; 40; 50; 60; 70; 80; 90; 100] : _ i32 _)
    |> am.iter fun s =>
        inl s = .^(0, s)
        inl result = zeta s
        result |> re |> _assert_ne 0
        result |> im |> _assert_ne 0

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_imaginary_axis ()

zeta / s: Complex { re: 0.0, im: 10.0 } / result: Complex { re: 1.756468592974963, im: -0.10151198543617117 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.0, im: 20.0 } / result: Complex { re: -0.05387338122970365, im: -1.7306610438309435 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.0, im: 30.0 } / result: Complex { re: -1.265122080538538, im: -0.6023204105605986 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.0, im: 40.0 } / result: Complex { re: 0.6337972539673052, im: -2.394846950499272 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.0, im: 50.0 } / result: Complex { re: -1.4712125886253156, im: 0.1199802326850745 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.0, im: 60.0 } / result: Complex { re: 1.3090142318386198, im: 0.8539064694153011 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.0, im: 70.0 } / result: Complex { re: -1.3246136758869096, im: 2.5275896369822415 } / z: Complex { re: 0.0, im

## test_critical_strip

In [ ]:
inl test_critical_strip () = run_test (3u8, 2u8) fun zeta, gamma =>
    ;[
        .^(0.5, 14.134725)
        .^(0.75, 20.5)
        .^(1.25, 30.1)
        .^(0.25, 40.0)
        .^(1.0, 50.0)
    ]
    |> fun x => a x : _ i32 _
    |> am.iter fun s =>
        inl result = zeta s
        result |> re |> _assert_ne 0
        result |> im |> _assert_ne 0

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_critical_strip ()

zeta / s: Complex { re: 0.5, im: 14.134725 } / result: Complex { re: 1.767429841384921e-8, im: -1.1102028930923147e-7 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.75, im: 20.5 } / result: Complex { re: 0.29344657597687224, im: -0.49494246097337435 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 1.25, im: 30.1 } / result: Complex { re: 0.5612352676213629, im: -0.3708223296098184 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.25, im: 40.0 } / result: Complex { re: 0.7344057041679518, im: -1.5656813889303969 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 1.0, im: 50.0 } / result: Complex { re: 0.44103873082309397, im: 0.281582455029683 } / z: Complex { re: 0.0, im: 0.0 }


## test_reflection_formula_for_specific_value

In [ ]:
inl test_reflection_formula_for_specific_value () = run_test (3u8, 2u8) fun zeta, gamma =>
    ;[
        .^(3, 4)
        .^(2.5, -3.5)
        .^(1.5, 2.5)
        .^(0.5, 14.134725)
    ]
    |> fun x => a x : _ i32 _
    |> am.iter fun s =>
        inl lhs = zeta s
        inl reflection_coefficient =
            (.^(2, 0) .** s)
            .* (.^(pi, 0) .** (s .- .^(1, 0)))
            .* (.^(pi, 0) .* s ./ .^(2, 0) |> complex_sin)
            .* gamma (.^(1, 0) .- s)

        inl one_minus_s = .^(1 - re s, -(im s))
        inl rhs = reflection_coefficient .* zeta one_minus_s

        re lhs - re rhs |> abs |> _assert_lt 0.0001
        im lhs - im rhs |> abs |> _assert_lt 0.0001

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_reflection_formula_for_specific_value ()

zeta / s: Complex { re: 3.0, im: 4.0 } / result: Complex { re: 0.8905549069650732, im: -0.00807594542432726 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -2.0, im: -4.0 } / result: Complex { re: 0.1659695435086394, im: -0.28741334644619076 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 2.5, im: -3.5 } / result: Complex { re: 0.8495541063184239, im: 0.048420327100649994 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -1.5, im: 3.5 } / result: Complex { re: 0.23243413923384182, im: 0.1737283788306166 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 1.5, im: 2.5 } / result: Complex { re: 0.7250357382596239, im: -0.21336341253664076 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: -0.5, im: -2.5 } / result: Complex { re: 0.29882548773595197, im: 0.06891862529873917 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 0.5, im: 14.134725 } / result: Complex { re: 1.767429841384921e-8, im: -1.1102028930923147e-7 } / z: Comple

## test_euler_product_formula

In [ ]:
inl test_euler_product_formula () = run_test (3u8, 2u8) fun zeta, gamma =>
    inl s_values = ;[2; 2.5; 3; 3.5; 4; 4.5; 5]
    inl primes = ;[2; 3; 5; 7; 11; 13; 17; 19; 23; 29; 31; 37; 41; 43; 47; 53; 59; 61; 67; 71]
    (a s_values : _ i32 _)
    |> am.iter fun s_re =>
        inl s = .^(s_re, 0)
        inl product =
            (1, (a primes : _ i32 _))
            ||> am.fold fun acc x =>
                acc * 1 / (1 - x ** -s_re)

        inl result = zeta s
        re result - product |> abs |> _assert_lt 0.01
        result |> im |> _assert_lt 0.01

()



In [ ]:
// // test
// // rust=
// // print_code=false

types ()
test_euler_product_formula ()

zeta / s: Complex { re: 2.0, im: 0.0 } / result: Complex { re: 1.6449340668482264, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 2.5, im: 0.0 } / result: Complex { re: 1.341487257250917, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 3.0, im: 0.0 } / result: Complex { re: 1.2020569031595942, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 3.5, im: 0.0 } / result: Complex { re: 1.1267338673170566, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 4.0, im: 0.0 } / result: Complex { re: 1.0823232337111381, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 4.5, im: 0.0 } / result: Complex { re: 1.0547075107614543, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }
zeta / s: Complex { re: 5.0, im: 0.0 } / result: Complex { re: 1.03692775514337, im: 0.0 } / z: Complex { re: 0.0, im: 0.0 }


## tests

In [ ]:
inl tests () =
    !\($'"}//"') : ()

    !\($'"#[test] fn test_zeta_at_known_values_() { //"') : ()
    test_zeta_at_known_values_ ()
    !\($'"} #[test] fn test_zeta_at_2_minus2() { //"') : ()
    test_zeta_at_2_minus2 ()
    !\($'"} #[test] fn test_trivial_zero_at_negative_even___() { //"') : ()
    test_trivial_zero_at_negative_even___ ()
    !\($'"} #[test] fn test_non_trivial_zero___() { //"') : ()
    test_non_trivial_zero___ ()
    !\($'"} #[test] fn test_real_part_greater_than_one___() { //"') : ()
    test_real_part_greater_than_one___ ()
    !\($'"} #[test] fn test_zeta_at_1___() { //"') : ()
    test_zeta_at_1___ ()
    !\($'"} #[test] fn test_symmetry_across_real_axis___() { //"') : ()
    test_symmetry_across_real_axis___ ()
    !\($'"} #[test] fn test_behavior_near_origin___() { //"') : ()
    test_behavior_near_origin___ ()
    !\($'"} #[test] fn test_zeta_at_minus_1() { //"') : ()
    test_zeta_at_minus_1 ()
    !\($'"} #[test] fn test_imaginary_axis() { //"') : ()
    test_imaginary_axis ()
    !\($'"} #[test] fn test_critical_strip() { //"') : ()
    test_critical_strip ()
    !\($'"} #[test] fn test_reflection_formula_for_specific_value() { //"') : ()
    test_reflection_formula_for_specific_value ()
    !\($'"} #[test] fn test_euler_product_formula() { //"') : ()
    test_euler_product_formula ()

()



In [ ]:
// // rust=

inl main (_args : array_base string) =
    inl value = 1i32
    console.write_line ($"$\"value: {!value}\"" : string)
    0i32

inl main () =
    types ()
    $"let tests () = !tests ()" : ()
    $"let main args = !main args" : ()


.fsx:
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>] type Func0<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>] type Func0<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>] type Box<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>] type Dyn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>] type Fn<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>] type FnUnit = class end
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>] type FnOnce<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>] type ActionFn2<'T, 'U> = class end
[<Fable.Core.Erase; Fable.Core.Emit("impl $0")>] type Impl<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("mut $0")>] type Mut<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&$0")>] type Ref<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("&'static $0")>] type StaticRef<'T> = class end
[<Fable.Core.Erase; Fable.Core.Emit("MutCell<$0>")>] type MutCe